# Media Analyser - By Gabriel Hanich

In [1]:
# Imports
import json
from collections import Counter
import matplotlib.pyplot as plt
from datetime import date, datetime
import datetime as dt

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import pr

import sys
sys.path.append('../') # Allows importing of local modules

from lib.data import getData
from lib.media import outlet, article

In [2]:
%matplotlib qt

In [3]:
# Constants
dataVersion = 2
stopwords = stopwords.words("english")

In [4]:
def daterange(date1, date2): # Returns list of dates between 2 dates
    for n in range(int((date2 - date1).days)+1):
        yield date1 + dt.timedelta(n)

In [5]:
with open("../Media Outlets.csv") as outletsFile: # Get list of outlets
    outletsData = getData(outletsFile)

outlets = []
for thisOutlet in outletsData:
    outlets.append(outlet(thisOutlet[0]))

    with open("../data/" + str(dataVersion) + "/json/" + thisOutlet[0] + ".json", encoding="utf-8") as dataFile:
        outlets[-1].setDayDict(json.load(dataFile)) # Write daydict to outlet Object

In [6]:
def getXYVals(thisOutlet, statType, startDate, endDate, titleSearchWords, commonWordCount):
    # Returns a list containing the x, y and z (annotation) for a given media outlet
    xVals = []
    yVals = []
    zVals = []
    for myDate in list(daterange(startDate, endDate)): # For every date between the given start and end date
        try:
            articleCount = 0
            avgSentiment = 0
            commonWords = []
            totalWords = []
            for articleIndex, article in enumerate(thisOutlet.dayDict[myDate.strftime("%d/%m/%Y")]):
                # If there are no title search words OR title contains wanted word
                if titleSearchWords == [] or set(titleSearchWords).intersection(word_tokenize(article["headline"].lower())):
                    articleCount += 1
                    avgSentiment += article["intensityScore"]
                    for word in word_tokenize(article["headline"]):
                        if len(word) > 2:
                            if word.lower() not in stopwords:
                                totalWords.append(str(word))

            commonWords = Counter(totalWords).most_common(commonWordCount)
            # Append data depending on chosen chart type        
            if statType == "articleCount":
                yVals.append(articleCount)
            elif statType == "avgSentiment":
                try:
                    yVals.append(avgSentiment / articleCount)
                except ZeroDivisionError:
                    yVals.append(0)
            elif statType  == "commonWords":
                yVals.append(articleCount)
                zVals.append(commonWords)

            xVals.append(myDate)
        except KeyError: # In case outlet has no articles published on that date (Returns empty lists)
            pass
    return xVals, yVals, zVals

In [13]:
# User settings
startingDate = "13/9/2021"
endDate = "23/01/2022"

doSetDates = True # Whether to use user-specified dates
doSetOutlets = False # Whether to limit search to a set of outlets
commonWordCount = 1 # How many common words per day are displayed
doTotalOutlets = False # Whether to take the outlets as a total
titleSearchWords = [] # Only include articles with headlines containing these words (Leave BLANK to disable)
setOutlets = ["ABC News"] # Only include a set of outlets
chartType = "articleCount" # Type of chart to make

In [14]:
startingDate = datetime.strptime(startingDate, "%d/%m/%Y")
endDate = datetime.strptime(endDate, "%d/%m/%Y")

In [15]:
# Do Total outlets
if doTotalOutlets:
    totalOutlet = outlet("Total")
    thisDayDict = {}
    for date in daterange(startingDate, endDate):
        strDate = datetime.strftime(date, "%d/%m/%Y")
        thisArticleList = []
        for mediaOutlet in outlets:
            try:
                for articleIndex, article in enumerate(mediaOutlet.dayDict[strDate]):
                    thisArticleList.append(article)
            except KeyError:
                pass
        if len(thisArticleList) != 0:
            thisDayDict[strDate] = thisArticleList
    totalOutlet.setDayDict(thisDayDict)

In [16]:
if doSetOutlets:
    # Trim down outlet list to only include those chosen
    newOutlets = []
    for outlet in outlets:
        if outlet.name in setOutlets:
            newOutlets.append(outlet)
    setOutlets = newOutlets


In [17]:
if doTotalOutlets:
    plotOutlets = [totalOutlet]
elif doSetOutlets:
    plotOutlets = setOutlets
else:
    plotOutlets = outlets

In [18]:
# Plot data 
fig, ax = plt.subplots(1)
ax.set_title(chartType)

for mediaOutlet in plotOutlets:
    try:
        if not doSetDates:
            startingDate = list(mediaOutlet.dayDict.keys())[0] # Get date of first article
            endDate = list(mediaOutlet.dayDict.keys())[-1] # Get date of last article

            startingDate = datetime.strptime(startingDate, "%d/%m/%Y")
            endDate = datetime.strptime(endDate, "%d/%m/%Y")

    except IndexError:
        continue
    
    doMultiCategories = False
    if len(titleSearchWords) != 0:
        if type(titleSearchWords[0]) == list:
            doMultiCategories = True
            for searchList in titleSearchWords:
                xVals, yVals, zVals = getXYVals(mediaOutlet, chartType, startingDate, endDate, searchList, commonWordCount)


                ax.plot(xVals, yVals, label=mediaOutlet.name + " " + str(searchList))  

                if len(zVals) != 0: # If there are annotations present
                    for dayIndex, dayPoint in enumerate(zVals):
                        for i in range(len(dayPoint)):
                            ax.annotate(dayPoint[i][0], (xVals[dayIndex], yVals[dayIndex] - i))
    if not doMultiCategories:
        xVals, yVals, zVals = getXYVals(mediaOutlet, chartType, startingDate, endDate, titleSearchWords, commonWordCount)
        ax.plot(xVals, yVals, label=mediaOutlet.name)  

        if len(zVals) != 0: # If there are annotations present
            for dayIndex, dayPoint in enumerate(zVals):
                for i in range(len(dayPoint)):
                    ax.annotate(dayPoint[i][0], (xVals[dayIndex], yVals[dayIndex] - i))
        
ax.legend()